In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix 
import pickle

In [2]:
X = pd.read_pickle('/kaggle/input/amex-imputed-aggregate-data/X_train_agg.pkl', compression='gzip') 
y = pd.read_pickle('/kaggle/input/amex-imputed-aggregate-data/y_train_agg.pkl', compression='gzip')             

In [3]:
#kaggle competition metric

def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

In [4]:
xgb_regressor = xgb.XGBRegressor() #create shell XGB Classifier model with default parameters

In [5]:
xgb_regressor = xgb_regressor.fit(X,y) #fit model with aggregate training data

In [7]:
predict = xgb_regressor.predict(X) #predict probably of credit default

In [13]:
acc = accuracy_score(y, predict.round())
print("Accuracy: %.2f%%" % (acc * 100.0))

0.9181413470527093


In [14]:
print(f'M = {amex_metric(pd.DataFrame(y), pd.DataFrame(predict, index=y.index, columns=["prediction"]))}')

M = 0.8392985836076163


In [16]:
print(classification_report(y, predict.round()))

              precision    recall  f1-score   support

           0       0.94      0.95      0.94    340085
           1       0.85      0.83      0.84    118828

    accuracy                           0.92    458913
   macro avg       0.90      0.89      0.89    458913
weighted avg       0.92      0.92      0.92    458913



In [17]:
pickle.dump(xgb_regressor, open('xgb_regressor_agg.sav', 'wb'))

In [18]:
X_test = pd.read_pickle('/kaggle/input/amex-imputed-aggregate-data/X_test_agg.pkl', compression='gzip')

In [20]:
model = pickle.load(open('/kaggle/working/xgb_regressor_agg.sav', 'rb'))
submission = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=['prediction'])

In [21]:
# index needs to be removed from submission csv
submission = submission.reset_index()
submission.to_csv('submission.csv', index=False)